# MNIST with SciKit-Learn and skorch

This notebooks shows how to define and train a simple Neural-Network with PyTorch and use it via skorch with SciKit-Learn.

<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/github/dnouri/skorch/blob/master/notebooks/Transfer_Learning.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/dnouri/skorch/blob/master/notebooks/Transfer_Learning.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

**Note**: If you are running this in [a colab notebook](https://colab.research.google.com/github/dnouri/skorch/blob/master/notebooks/Transfer_Learning.ipynb), we recommend you enable a free GPU by going:

> **Runtime**   →   **Change runtime type**   →   **Hardware Accelerator: GPU**

If you are running in colab, you should install the dependencies and download the dataset by running the following cell:

In [ ]:
! [ ! -z "$COLAB_GPU" ] && pip install torch scikit-learn==0.20.* skorch

In [5]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np

## Loading Data
Using SciKit-Learns ```fetch_mldata``` to load MNIST data.

In [9]:
mnist = fetch_openml('mnist_784', data_home='datasets')

In [11]:
mnist.data.shape

(70000, 784)

## Preprocessing Data

Each image of the MNIST dataset is encoded in a 784 dimensional vector, representing a 28 x 28 pixel image. Each pixel has a value between 0 and 255, corresponding to the grey-value of a pixel.<br />
The above ```featch_mldata``` method to load MNIST returns ```data``` and ```target``` as ```uint8``` which we convert to ```float32``` and ```int64``` respectively.

In [12]:
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

As we will use ReLU as activation in combination with softmax over the output layer, we need to scale `X` down. An often use range is [0, 1].

In [13]:
X /= 255.0

In [14]:
X.min(), X.max()

(0.0, 1.0)

Note: data is not normalized.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
assert(X_train.shape[0] + X_test.shape[0] == mnist.data.shape[0])

In [18]:
X_train.shape, y_train.shape

((52500, 784), (52500,))

## Build Neural Network with Torch
Simple, fully connected neural network with one hidden layer. Input layer has 784 dimensions (28x28), hidden layer has 98 (= 784 / 8) and output layer 10 neurons, representing digits 0 - 9.

In [19]:
import torch
from torch import nn
import torch.nn.functional as F

In [20]:
torch.manual_seed(0);
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
mnist_dim = X.shape[1]
hidden_dim = int(mnist_dim/8)
output_dim = len(np.unique(mnist.target))

In [22]:
mnist_dim, hidden_dim, output_dim

(784, 98, 10)

A Neural network in PyTorch's framework.

In [23]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=mnist_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

Skorch allows to use PyTorch's networks in the SciKit-Learn setting.

In [26]:
from skorch import NeuralNetClassifier

In [27]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device=device,
)

In [28]:
net.fit(X_train, y_train);

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.8338       0.8872        0.3968  1.4746
      2        0.4333       0.9092        0.3143  1.3837
      3        0.3603       0.9214        0.2749  1.3581
      4        0.3200       0.9326        0.2322  1.3734
      5        0.2942       0.9357        0.2178  1.4722
      6        0.2731       0.9413        0.2000  1.3952
      7        0.2550       0.9462        0.1878  1.2796
      8        0.2418       0.9484        0.1765  1.2885
      9        0.2337       0.9487        0.1732  1.3058
     10        0.2246       0.9540        0.1602  1.3167
     11        0.2137       0.9543        0.1532  1.2999
     12        0.2085       0.9562        0.1489  1.2814
     13        0.1985       0.9582        0.1435  1.2777
     14        0.1973       0.9571        0.1411  1.2860
     15        0.1930       0.9595        0.1357  1.3050
     16        0.1847       0.9

## Prediction

In [29]:
predicted = net.predict(X_test)

In [30]:
np.mean(predicted == y_test)

0.9628

An accuracy of nearly 96% for a network with only one hidden layer is not too bad

# Convolutional Network
PyTorch expects a 4 dimensional tensor as input for its 2D convolution layer. The dimensions represent:
* Batch size
* Number of channel
* Height
* Width

As initial batch size the number of examples needs to be provided. MNIST data has only one channel. As stated above, each MNIST vector represents a 28x28 pixel image. Hence, the resulting shape for PyTorch tensor needs to be (x, 1, 28, 28). 

In [31]:
XCnn = X.reshape(-1, 1, 28, 28)

In [32]:
XCnn.shape

(70000, 1, 28, 28)

In [33]:
XCnn_train, XCnn_test, y_train, y_test = train_test_split(XCnn, y, test_size=0.25, random_state=42)

In [34]:
XCnn_train.shape, y_train.shape

((52500, 1, 28, 28), (52500,))

In [35]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1600, 128) # 1600 = number channels * width * height
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3)) # flatten over channel, height and width = 1600
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.softmax(x, dim=-1)
        return x

In [36]:
cnn = NeuralNetClassifier(
    Cnn,
    max_epochs=15,
    lr=1,
    optimizer=torch.optim.Adadelta,
    device=device,
)

In [37]:
cnn.fit(XCnn_train, y_train);

In [28]:
cnn_pred = cnn.predict(XCnn_test)

In [29]:
np.mean(cnn_pred == y_test)

0.9912571428571428

An accuracy of 99.1% should suffice for this example!